In [10]:
import os
from bs4 import BeautifulSoup
import bs4
import numpy as np
import requests,json,time,random,logging,math
import pandas as pd
import logging
logging.basicConfig(level=logging.INFO)

In [11]:
with open('dist/directory.json') as f:
    directory = json.load(f)

In [12]:
taken_keys = set(directory[0].keys())
parsed = []
for i,person in enumerate(directory):
    with open(f'people_html/{person["id"]}.html') as f:
        soup = BeautifulSoup(f.read(),'html')
    container = soup.find('div',class_="docbox_horizontal")
    if container is None and 'document not found' in soup.text.lower():
        logging.error(f"Missing Document {i} - {person['name']} ({person['id']}) file(people_html/{person['id']}.html)")
        parsed.append(person | {'no_data': True})
        continue
    elif container is None:
        print(soup)
        break
    data = container.find_all('span','text')[1]
    bio = {}
    key = None
    value = None
    for child in data.children:
        if isinstance(child,bs4.element.Tag) and child.name=="strong":
            title = child.text.strip().lower().replace(' ','_').replace(':','')
        if title is not None and isinstance(child,bs4.element.NavigableString):
            value = child.text.strip()
            bio[key] = value
        elif title is not None:
            key = title
        else:
            print(child.name,type(child))
    if len(set(bio.keys()) & taken_keys)==0:
        parsed.append(person | bio)
        #print(f"Paresd {i} - {person['name']} ({person['id']})")
    else:
        break
    
logging.info(f"Sucessfully finished parsing {len(directory)} people.")
with open('dist/cqpress.json','w+') as f:
    json.dump(parsed, f)

ERROR:root:Missing Document 34 - Alexander, Rodney (pia113-Alexander_Rodney) file(people_html/pia113-Alexander_Rodney.html)
ERROR:root:Missing Document 35 - Alexander, Rodney (pia113-Alexander_Rodney) file(people_html/pia113-Alexander_Rodney.html)
ERROR:root:Missing Document 280 - Bishop, Timothy H. (pia113-Bishop_Timothy_H.) file(people_html/pia113-Bishop_Timothy_H..html)
ERROR:root:Missing Document 374 - Breaux, John B. (pia108-669522) file(people_html/pia108-669522.html)
ERROR:root:Missing Document 375 - Breaux, John B. (pia108-669522) file(people_html/pia108-669522.html)
ERROR:root:Missing Document 858 - D'Amours, Norman E. (bdac96-0453054588) file(people_html/bdac96-0453054588.html)
ERROR:root:Missing Document 865 - Danielson, George Elmore (bdac96-0453054591) file(people_html/bdac96-0453054591.html)
ERROR:root:Missing Document 910 - Delahunt, William D. (Bill) (pia111-Delahunt-Bill) file(people_html/pia111-Delahunt-Bill.html)
ERROR:root:Missing Document 960 - Dingell, John David,

In [13]:
df = pd.DataFrame(parsed)
df = df.drop_duplicates(subset='id')
print(df.shape)
df.to_excel("dist/cqpress.xlsx")
df = df[df['military_service'].notna()]
df = df[~df['military_service'].str.contains("Did not serve") & ~df['military_service'].str.match(";")]
df.head(1)

(3629, 17)


,id,link,name,party,state,affilication,term,born,died,religion,education,political_background,occupations,election_record,occupation,military_service,no_data
5,bdac96-581,document.php?id=bdac96-581,"Abele, Homer E.",R,Ohio,U.S. Representative,1963-1965,"November 21, 1916","May 12, 2000",Not specified,"Attended Ohio University, 1946–1948; LL.B. fro...","Ohio General Assembly, 1949–1952; delegate to ...",Lawyer; public service; judge,1962=52%,NaN,Air Corps 1943-1946,NaN


In [21]:
df['military_service'] = df['military_service'].str.replace(';','')
df['military_service'].to_list()[:5]

['Army National Guard 1971-1977',
 'Army 1970-1971 Michigan National Guard 1990-2009',
 'U.S. Marine Corps Reserve  1970-1975',
 'Army 1970-1971',
 'Army 1973-1978']

In [15]:
df = df.drop(['id','link','party','state','affilication','term','religion','education','political_background','occupations','election_record','occupation','no_data'],axis=1)

In [16]:
df.head()

,name,born,died,military_service
5,"Abele, Homer E.","November 21, 1916","May 12, 2000",Air Corps 1943-1946
8,"Abourezk, James George","February 24, 1931",NaN,Navy 1948-1952
10,"Abraham, Ralph","September 16, 1954",NaN,Army 1986-1989
14,"Adair, Edwin Ross","December 14, 1907","May 5, 1983",Army 1941-1945
16,"Adams, Brockman (Brock)","January 13, 1927","September 10, 2004",Navy 1944-1946


In [17]:
military_service = df['military_service'].str.extractall("(\d{4}-\d{4})|(\d{4}-present)")

In [18]:
df['military_durations'] = military_service.groupby(level=0)[0].apply(list)
df = df[df['military_durations'].notna()]
df.to_excel("dist/military_durations.xlsx")

In [19]:
def has_date_in_draft(durations):
    for duration in durations:
        try:
            if isinstance(duration,str) or not math.isnan(duration):
                start,end = duration.split('-')
                return int(start) in range(1970,1974)
        except Exception as e:
            print(duration)
            raise e
    return False

In [20]:
df['is_during_draft'] = df['military_durations'].map(has_date_in_draft)
df = df[df['is_during_draft']]
df.to_excel('dist/people_during_draft.xlsx')